<a href="https://colab.research.google.com/github/marybello/optimizing-hyperparamters/blob/main/hyperparametertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from hyperopt import tpe, hp, fmin, STATUS_OK, Trials
from hyperopt.pyll.base import scope
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf


In [ ]:
train_path = tf.keras.utils.get_file('iris_training.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_test.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

In [ ]:
column_names = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
species =['Setosa','Versicolor','Virginica']
train_df = pd.read_csv(train_path, names=column_names, header=0)
test_df = pd.read_csv(test_path, names=column_names, header=0)

In [ ]:
train_df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
x_train = train_df.iloc[:,0:4]
y_train = train_df.iloc[:,4]
x_test = test_df.iloc[:,0:4]
y_test = test_df.iloc[:,4]

In [ ]:
X = pd.concat([x_train,x_test])
y = y_train.append(y_test)

In [ ]:
#Hyperopt tuning
space = {
    "n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
    "max_depth": hp.quniform("max_depth", 1, 15,1),
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
}

In [ ]:
#objective function
def hyperparameter_tuning(params):
    clf = RandomForestClassifier(**params,n_jobs=-1)
    acc = cross_val_score(clf, X, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [ ]:
# Initialize trials object
trials = Trials()
#optimizer
best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████| 100/100 [03:38<00:00,  2.19s/it, best loss: -0.9666666666666668]
Best: {'criterion': 0, 'max_depth': 5.0, 'n_estimators': 0}


In [ ]:
trials.results

[{'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.96, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9666666666666668, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.96, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.96, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9666666666666666, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.9533333333333334, 'status': 'ok'},
 {'loss': -0.96, 'status': 'ok'},
 {'loss': -0.96, 'status': 'ok'},
 {'loss': -0.9466666666666667, 'status': 'ok'},
 {'loss': -0.9

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 4.9MB/s 


In [ ]:
#Scikit Optimize
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical,Integer
from skopt.utils import use_named_args
from skopt import gp_minimize


In [ ]:
# define search space 
params = {
    "n_estimators": [100, 200, 300, 400],
    "max_depth": (1, 9),
    "criterion": ["gini", "entropy"],
}

In [ ]:
rf_classifier=RandomForestClassifier()

In [ ]:
# define the search
search = BayesSearchCV(
    estimator=rf_classifier,
    search_spaces=params,
    n_jobs=1,
    cv=5,
    n_iter=30,
    scoring="accuracy",
    verbose=4,
    random_state=42
)

In [ ]:
search.fit(X,y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] criterion=entropy, max_depth=7, n_estimators=400 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=entropy, max_depth=7, n_estimators=400, score=0.900, total=   1.2s
[CV] criterion=entropy, max_depth=7, n_estimators=400 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV]  criterion=entropy, max_depth=7, n_estimators=400, score=1.000, total=   0.6s
[CV] criterion=entropy, max_depth=7, n_estimators=400 ................


...

[CV]  criterion=gini, max_depth=3, n_estimators=400, score=0.900, total=   0.6s
[CV] criterion=gini, max_depth=3, n_estimators=400 ...................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.7s remaining:    0.0s


[CV]  criterion=gini, max_depth=3, n_estimators=400, score=1.000, total=   0.6s
[CV] criterion=gini, max_depth=3, n_estimators=400 ...................
[CV]  criterion=gini, max_depth=3, n_estimators=400, score=0.967, total=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.9s finished


BayesSearchCV(cv=5, error_score='raise',
              estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                               class_weight=None,
                                               criterion='gini', max_depth=None,
                                               max_features='auto',
                                               max_leaf_nodes=None,
                                               max_samples=None,
                                               min_impurity_decrease=0.0,
                                               min_impurity_split=None,
                                               min_samples_leaf=1,
                                               min_samples_split=2,
                                               min_weight_fraction_leaf=0.0,
                                               n_estimators=100, n_jobs=-1,
                                               oob_score=False,
                                   

In [ ]:
print(search.best_score_)
print(search.best_params_)

0.9666666666666667
OrderedDict([('criterion', 'gini'), ('max_depth', 3), ('n_estimators', 400)])


In [ ]:
!pip install optuna

     |████████████████████████████████| 256kB 8.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 163kB 15.4MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 112kB 22.2MB/s 
     |████████████████████████████████| 143kB 22.3MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
  Created wheel for optuna: filename=optuna-2.2.0-cp36-none-any.whl size=340928 sha256=4a7bcb455965acc7fe156611770075fcef49a192b7b6a34da162f9fa12988ee1
  Stored in directory: /root/.cache/pip/wheels/a6/27/26/3ef57a8fb2c8bceef3f4ece73483cec1895e06eae442d34a41
Successfully built optuna
  Created wheel for pyperclip: filename=pyperclip-1.8.0-cp36-none-any.whl size=8693 sha256=9a4445cabf3f6f7c55e366502b86edbd85090b8c4e63b12f9f67801d16be0241
  Stored in directory: /root/.cache/pip/wheels/b

In [ ]:
#Optuna in practice
import joblib
import optuna
from optuna.samplers import TPESampler


In [ ]:
# define the search space and the objecive function
def objective(trial):
    # Define the search space
    criterions = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depths = trial.suggest_int('max_depth', 1, 9, 1)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 100)

    clf = RandomForestClassifier(n_estimators=n_estimators,
                                 criterion=criterions,
                                 max_depth=max_depths,
                                 n_jobs=-1)
    score = cross_val_score(clf, X, y, scoring="accuracy").mean()

    return score

In [ ]:
# create a study object 

study = optuna.create_study(study_name="randomForest_optimization",
                            direction="maximize", #can be maximize or minimize
                            sampler=TPESampler())

[I 2020-10-06 12:49:15,540] A new study created in memory with name: randomForest_optimization


In [ ]:
#optimize the study
study.optimize(objective, n_trials=10)


[I 2020-10-06 12:49:56,086] Trial 0 finished with value: 0.96 and parameters: {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 800}. Best is trial 0 with value: 0.96.
[I 2020-10-06 12:50:01,570] Trial 1 finished with value: 0.96 and parameters: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 800}. Best is trial 0 with value: 0.96.
[I 2020-10-06 12:50:06,456] Trial 2 finished with value: 0.9466666666666667 and parameters: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 700}. Best is trial 0 with value: 0.96.
[I 2020-10-06 12:50:12,004] Trial 3 finished with value: 0.9533333333333334 and parameters: {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 800}. Best is trial 0 with value: 0.96.
[I 2020-10-06 12:50:14,361] Trial 4 finished with value: 0.96 and parameters: {'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 300}. Best is trial 0 with value: 0.96.
[I 2020-10-06 12:50:15,571] Trial 5 finished with value: 0.8266666666666665 and parameters: {'criterion

In [ ]:
print(study.best_params)
print(study.best_value)

{'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 800}
0.96


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)